# Documentação: Coleta de Dados de Usuários do GitHub

## Introdução
Este script realiza a coleta de informações sobre usuários do GitHub de forma assíncrona, utilizando a API do GitHub. Ele busca apenas chaves relevantes e salva os dados processados em um arquivo CSV.

## Dependências
Antes de executar o script, certifique-se de instalar as bibliotecas necessárias:

```bash
pip install aiohttp nest_asyncio pandas tqdm
```

## Estrutura do Script

### 1. Configuração do Loop Assíncrono
```python
nest_asyncio.apply()
```
Em ambientes como Jupyter Notebook, é necessário aplicar `nest_asyncio` para evitar conflitos com o loop de eventos assíncrono.

### 2. Definição das Chaves Relevantes
```python
chaves_relevantes = [
    "login", "id", "name", "company", "blog", "location", "email", "bio",
    "twitter_username", "public_repos", "public_gists", "followers", "following",
    "created_at", "updated_at"
]
```
Define quais informações serão extraídas da API do GitHub.

### 3. Função `fetch_user`
```python
async def fetch_user(session, user_id, semaphore, timeout=1000):
```
Essa função busca os dados de um usuário específico no GitHub, respeitando um limite de requisições concorrentes.

**Parâmetros:**
- `session`: Instância de `aiohttp.ClientSession`.
- `user_id`: ID do usuário do GitHub.
- `semaphore`: Controle de concorrência.
- `timeout`: Tempo limite para a requisição.

**Retorno:**
- Dicionário contendo apenas as chaves relevantes ou `None` em caso de erro.

### 4. Função `get_users_data_async`
```python
async def get_users_data_async(id_list, max_concurrent_requests=10):
```
Essa função gerencia a execução assíncrona para buscar informações de múltiplos usuários.

**Parâmetros:**
- `id_list`: Lista de IDs de usuários do GitHub.
- `max_concurrent_requests`: Número máximo de requisições concorrentes (padrão: 10).

**Retorno:**
- Lista de dicionários contendo os dados dos usuários.

### 5. Função `get_users_data`
```python
def get_users_data(id_list):
```
Função wrapper para chamar a versão assíncrona de maneira síncrona.

**Parâmetros:**
- `id_list`: Lista de IDs de usuários do GitHub.

**Retorno:**
- Lista de dicionários contendo os dados dos usuários.

### 6. Carregamento de IDs e Salvamento dos Dados
```python
users_ids = pd.read_csv(f'{base_path}/users_ids.csv', encoding='utf-8')
users_ids = users_ids['user_id'].to_list()
```
Os IDs dos usuários são lidos de um arquivo CSV, e os dados extraídos são salvos em um novo arquivo CSV no caminho definido por `base_path`.

**Fluxo Completo:**
1. Lê a lista de IDs de usuários do GitHub a partir de um CSV.
2. Executa a busca de dados de forma assíncrona.
3. Filtra os dados mantendo apenas as chaves relevantes.
4. Salva os dados extraídos em um arquivo CSV.

### 7. Uso de Variáveis de Ambiente
O script requer duas variáveis de ambiente:
- `GITHUB_TOKEN`: Token de acesso à API do GitHub.
- `DIR_NAME`: Nome do diretório base para armazenamento dos dados processados.

Certifique-se de defini-las antes de executar o script.


# Main

In [1]:
import os
import asyncio
import aiohttp
import pandas as pd
from tqdm.asyncio import tqdm
import nest_asyncio

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Patch the current event loop (útil em ambientes como Jupyter Notebook)
nest_asyncio.apply()

# Lista de chaves relevantes a serem coletadas na resposta da API do GitHub.
chaves_relevantes = [
    "login",
    "id",
    "name",
    "company",
    "blog",
    "location",
    "email",
    "bio",
    "twitter_username",
    "public_repos",
    "public_gists",
    "followers",
    "following",
    "created_at",
    "updated_at"
]

async def fetch_user(session, user_id, semaphore, timeout=1000):
    """
    Busca os dados de um usuário do GitHub e retorna apenas as chaves relevantes.
    
    Args:
        session (aiohttp.ClientSession): A sessão HTTP ativa.
        user_id (int ou str): ID do usuário no GitHub.
        semaphore (asyncio.Semaphore): Semáforo para limitar requisições concorrentes.
        timeout (int): Tempo máximo de espera para a requisição.
        
    Returns:
        dict ou None: Dicionário com os dados filtrados do usuário ou None em caso de falha.
    """
    url = f"https://api.github.com/user/{user_id}"
    async with semaphore:
        try:
            async with session.get(url, timeout=timeout) as response:
                if response.status == 200:
                    data = await response.json()
                    # Filtra os dados, mantendo apenas as chaves relevantes.
                    filtered_data = { chave: data.get(chave) for chave in chaves_relevantes }
                    return filtered_data
                else:
                    print(f"Erro na requisição para o usuário {user_id}: {response.status}")
                    return None
        except asyncio.TimeoutError:
            print(f"Tempo esgotado na requisição para o usuário {user_id}")
            return None

async def get_users_data_async(id_list, max_concurrent_requests=10):
    """
    Recupera de forma assíncrona os dados dos usuários do GitHub,
    mantendo apenas as chaves relevantes.
    
    Args:
        id_list (list): Lista de IDs de usuários do GitHub.
        max_concurrent_requests (int): Número máximo de requisições HTTP concorrentes.
        
    Returns:
        list: Lista de dicionários com os dados filtrados dos usuários.
    """
    token = os.getenv('GITHUB_TOKEN')
    if not token:
        raise ValueError("A variável de ambiente GITHUB_TOKEN não está definida.")
    
    headers = {
        'Authorization': f'Token {token}',
        'Accept': 'application/vnd.github.v3+json'
    }
    
    semaphore = asyncio.Semaphore(max_concurrent_requests)
    
    async with aiohttp.ClientSession(headers=headers) as session:
        tasks = [fetch_user(session, user_id, semaphore) for user_id in id_list]
        results = []
        for coro in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Processing"):
            user_data = await coro
            if user_data:
                results.append(user_data)
        return results

def get_users_data(id_list):
    """
    Wrapper síncrono para executar a recuperação assíncrona dos dados dos usuários.
    
    Args:
        id_list (list): Lista de IDs de usuários do GitHub.
    
    Returns:
        list: Lista de dicionários com os dados filtrados dos usuários.
    """
    return asyncio.run(get_users_data_async(id_list))


In [3]:
dir_name = os.getenv('DIR_NAME')
base_path = f'../data/{dir_name}'
os.makedirs(base_path, exist_ok=True)

users_ids = pd.read_csv(f'{base_path}/users_ids.csv', encoding='utf-8')
users_ids = users_ids['user_id'].to_list()

filename = f"{base_path}/users_data.csv"
users_data = get_users_data(users_ids)

Processing: 100%|██████████| 1896/1896 [01:55<00:00, 16.43it/s]


In [4]:
users_data = pd.DataFrame(users_data)
users_data.to_csv(filename, index=False, encoding='utf-8')

print(f'Gravados dados de {len(users_data)} usuários.')

Gravados dados de 1896 usuários.
